In [1]:
# !pip install deeprobust
# !conda install pytorch torchvision torchaudio -c pytorch
import torch
# print(torch.__version__)
!pip install torch-geometric
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import watts_strogatz_graph
from networkx.generators.community import random_partition_graph

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis

import random



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.4 MB/s eta 0:00:00


In [2]:
import os
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import torch
from torch import Tensor
import torch_geometric
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid
import networkx as nx
from networkx.algorithms import community
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

In [3]:
import numpy
import torch
%load_ext autoreload
%autoreload 2

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))



from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import watts_strogatz_graph
from networkx.generators.community import random_partition_graph
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from random import sample

In [5]:
# from deeprobust.graph.data import Dataset

In [6]:
from scipy.sparse import csr_matrix
from scipy.sparse import csgraph
from scipy.sparse.linalg import inv

In [7]:
import os
os.getcwd()

'/content'

In [8]:
dataset = os.path.join(os.getcwd(),'Cora')
dataset

'/content/Cora'

In [9]:
import torch
from torch_geometric.datasets import Coauthor, Planetoid, CitationFull
from torch_geometric.utils import to_dense_adj
import numpy as np

seed=42
torch.manual_seed(seed)
# Load dataset
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0]

X = data.x.to_dense()
adj = to_dense_adj(data.edge_index)[0]
labels = data.y.numpy()

# Number of nodes
N = data.num_nodes
NO_OF_CLASSES = len(torch.unique(data.y))

# Calculate split sizes
train_size = int(0.8 * N)
val_size = int(0.1 * N)
test_size = N - train_size - val_size

# Create random permutation of indices
indices = np.random.permutation(N)
train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + val_size]
test_indices = indices[train_size + val_size:]

# Initialize masks
data.train_mask = torch.zeros(N, dtype=torch.bool)
data.val_mask = torch.zeros(N, dtype=torch.bool)
data.test_mask = torch.zeros(N, dtype=torch.bool)

# Set mask values
data.train_mask[train_indices] = True
data.val_mask[val_indices] = True
data.test_mask[test_indices] = True

# Verifying the split
print(f"Number of training nodes: {data.train_mask.sum().item()}")
print(f"Number of validation nodes: {data.val_mask.sum().item()}")
print(f"Number of test nodes: {data.test_mask.sum().item()}")

adj = adj[data.train_mask][:, data.train_mask]

# Accessing the train, validation, and test sets
X = data.x[data.train_mask]
X_val = data.x[data.val_mask]
X_test = data.x[data.test_mask]

labels = data.y[data.train_mask]
labels_val = data.y[data.val_mask]
labels_test = data.y[data.test_mask]

print(f"Train shapes - X: {X.shape}")
print(f"Val shapes - X: {X_val.shape}")
print(f"Test shapes - X: {X_test.shape}")

Processing...


Number of training nodes: 2166
Number of validation nodes: 270
Number of test nodes: 272
Train shapes - X: torch.Size([2166, 1433])
Val shapes - X: torch.Size([270, 1433])
Test shapes - X: torch.Size([272, 1433])


Done!


In [10]:
del dataset

In [11]:
def get_laplacian(adj):
    b=torch.ones(adj.shape[0])
    return torch.diag(adj@b)-adj

theta = get_laplacian(adj)
print(theta.shape)


torch.Size([2166, 2166])


In [12]:
# dataset_name = 'flickr'

# data = Dataset(root='', name=dataset_name, setting='gcn',seed=10)

# adj, features, labels = data.adj, data.features, data.labels
# idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

# theta = csgraph.laplacian(adj).tocsr()
features = X.numpy()
NO_OF_NODES = X.shape[0]
# NO_OF_CLASSES =  7


print(NO_OF_CLASSES,NO_OF_NODES)

7 2166


In [13]:
def convertScipyToTensor(coo):
  try:
    coo = coo.tocoo()
  except:
    coo = coo
  values = coo.data
  indices = np.vstack((coo.row, coo.col))

  i = torch.LongTensor(indices)
  v = torch.FloatTensor(values)
  shape = coo.shape

  return torch.sparse.FloatTensor(i, v, torch.Size(shape))

In [14]:
from scipy.sparse import random
from scipy.sparse.linalg import norm
from scipy.sparse import csr_matrix

p = X.shape[0]
k = int(p*0.05)
n = X.shape[1]
lambda_param = 100
beta_param = 50
alpha_param = 100
gamma_param = 100
lr = 1e-5
thresh = 1e-10

from scipy.sparse import random
from scipy.stats import rv_continuous
class CustomDistribution(rv_continuous):
    def _rvs(self,  size=None, random_state=None):
        return random_state.standard_normal(size)
temp = CustomDistribution(seed=1)
temp2 = temp()  # get a frozen version of the distribution
X_tilde = random(k, n, density=0.25, random_state=1, data_rvs=temp2.rvs)
C = random(p, k, density=0.25, random_state=1, data_rvs=temp2.rvs)



In [15]:
def experiment(lambda_param, beta_param, alpha_param, gamma_param, C, X_tilde, theta, X):
    p = X.shape[0]
    k = int(p*0.05)
    n = X.shape[1]
    ones = csr_matrix(np.ones((k,k)))
    ones = convertScipyToTensor(ones)
    ones = ones.to_dense()
    J = np.outer(np.ones(k), np.ones(k))/k
    J = csr_matrix(J)
    J = convertScipyToTensor(J)
    J = J.to_dense()
    zeros = csr_matrix(np.zeros((p,k)))
    zeros = convertScipyToTensor(zeros)
    zeros = zeros.to_dense()
    X_tilde = convertScipyToTensor(X_tilde)
    X_tilde = X_tilde.to_dense()
    C = convertScipyToTensor(C)
    C = C.to_dense()
    eye = torch.eye(k)

    try:
        theta = convertScipyToTensor(theta)
    except:
        theta = theta
    try:
        X = convertScipyToTensor(X)
        X = X.to_dense()
    except:
        X = X

    if(torch.cuda.is_available()):
        X_tilde = X_tilde.cuda()
        C = C.cuda()
        theta = theta.cuda()
        X = X.cuda()
        J = J.cuda()
        zeros = zeros.cuda()
        ones = ones.cuda()
        eye = eye.cuda()

    def relu(x):
        return torch.maximum(x, torch.zeros_like(x))

    def relu_derivative(x):
        return (x > 0).float()

    def update(X_tilde, C, i):
        global L
        thetaC = theta@C
        CT = torch.transpose(C,0,1)
        X_tildeT = torch.transpose(X_tilde,0,1)

        # Apply ReLU to CX_tilde
        CX_tilde_pre_relu = C@X_tilde
        CX_tilde = relu(CX_tilde_pre_relu)

        # Store the ReLU derivative mask for gradient computation
        relu_mask = relu_derivative(CX_tilde_pre_relu)

        relu_applied_grad = (CX_tilde - X) * relu_mask

        t1 = CT@thetaC + J
        term_bracket = torch.linalg.pinv(t1)
        thetacX_tilde = thetaC@(X_tilde)

        L = 1/k

        # Modified gradient terms to account for ReLU
        t1 = -2*gamma_param*(thetaC@term_bracket)
        # Modify t2 to account for ReLU derivative
        # t2 = alpha_param*(CX_tilde-X)@(X_tildeT) * relu_mask
        t2 = alpha_param * relu_applied_grad @ X_tildeT
        t3 = 2*thetacX_tilde@(X_tildeT)
        t4 = lambda_param*(C@ones)
        t5 = 2*beta_param*(thetaC@CT@thetaC)

        # Update C with modified gradient
        T2 = (t1+t2+t3+t4+t5)/L
        Cnew = (C-T2).maximum(zeros)

        # Update X_tilde
        t1 = CT@thetaC*(2/alpha_param)
        t2 = CT@C
        t1 = torch.linalg.pinv(t1+t2)
        t1 = t1@CT
        t1 = t1@X
        X_tilde_new = t1  # Apply ReLU to X_tilde_new

        # Apply thresholding and normalization
        Cnew[Cnew<thresh] = thresh
        for i in range(len(Cnew)):
            Cnew[i] = Cnew[i]/torch.linalg.norm(Cnew[i],1)
        for i in range(len(X_tilde_new)):
            X_tilde_new[i] = X_tilde_new[i]/torch.linalg.norm(X_tilde_new[i],1)

        return X_tilde_new, Cnew

    for i in tqdm(range(30)):
        X_tilde, C = update(X_tilde, C, i)

    return X_tilde, C

In [16]:
def experiment2(lambda_param,beta_param,alpha_param,gamma_param,C,X_tilde,theta,X):
      p = X.shape[0]
      k = int(p*0.7)
      n = X.shape[1]
      ones = csr_matrix(np.ones((k,k)))
      ones = convertScipyToTensor(ones)
      ones = ones.to_dense()
      J = np.outer(np.ones(k), np.ones(k))/k
      J = csr_matrix(J)
      J = convertScipyToTensor(J)
      J = J.to_dense()
      zeros = csr_matrix(np.zeros((p,k)))
      zeros = convertScipyToTensor(zeros)
      zeros = zeros.to_dense()
      X_tilde = convertScipyToTensor(X_tilde)
      X_tilde = X_tilde.to_dense()
      C = convertScipyToTensor(C)
      C = C.to_dense()
      eye = torch.eye(k)
      try:
        theta = convertScipyToTensor(theta)
      except:
        theta = theta
      try:
        X = convertScipyToTensor(X)
        X = X.to_dense()
      except:
        X = X

      if(torch.cuda.is_available()):
        # print("yes")
        X_tilde = X_tilde.cuda()
        C = C.cuda()
        theta = theta.cuda()
        X = X.cuda()
        J = J.cuda()
        zeros = zeros.cuda()
        ones = ones.cuda()
        eye = eye.cuda()

      def update(X_tilde,C,i):
          global L
          thetaC = theta@C
          CT = torch.transpose(C,0,1)
          X_tildeT = torch.transpose(X_tilde,0,1)
          CX_tilde = C@X_tilde
          t1 = CT@thetaC + J
          term_bracket = torch.linalg.pinv(t1)
          thetacX_tilde = thetaC@(X_tilde)
          CTX = CT@X

          L = 1/k

          t1 = -2*gamma_param*(thetaC@term_bracket)
          t2 = alpha_param*X@((X_tilde - CTX).T)
          # t3 = 2*thetacX_tilde@(X_tildeT)
          t3 = zeros
          t4 = lambda_param*(C@ones)
          t5 = 2*beta_param*(thetaC@CT@thetaC)
          T2 = (t1+t2+t3+t4+t5)/L
          Cnew = (C-T2).maximum(zeros)
          t1 = CT@thetaC*(2/alpha_param)
          t2 = CT@C
          t1 = torch.linalg.pinv(t1+t2)
          t1 = t1@CT
          t1 = t1@X
          X_tilde_new = CTX
          Cnew[Cnew<thresh] = thresh
          for i in range(len(Cnew)):
              Cnew[i] = Cnew[i]/torch.linalg.norm(Cnew[i],1)
          for i in range(len(X_tilde_new)):
            X_tilde_new[i] = X_tilde_new[i]/torch.linalg.norm(X_tilde_new[i],1)
          return X_tilde_new,Cnew


      for i in tqdm(range(10)):
          X_tilde,C = update(X_tilde,C,i)

      return X_tilde,C




In [17]:
def experiment3(lambda_param,beta_param,alpha_param,gamma_param,C,X_tilde,theta,X):
      p = X.shape[0]
      k = int(p*0.05)
      n = X.shape[1]
      ones = csr_matrix(np.ones((k,k)))
      ones = convertScipyToTensor(ones)
      ones = ones.to_dense()
      J = np.outer(np.ones(k), np.ones(k))/k
      J = csr_matrix(J)
      J = convertScipyToTensor(J)
      J = J.to_dense()
      zeros = csr_matrix(np.zeros((p,k)))
      zeros = convertScipyToTensor(zeros)
      zeros = zeros.to_dense()
      X_tilde = convertScipyToTensor(X_tilde)
      X_tilde = X_tilde.to_dense()
      C = convertScipyToTensor(C)
      C = C.to_dense()
      eye = torch.eye(k)
      try:
        theta = convertScipyToTensor(theta)
      except:
        theta = theta
      try:
        X = convertScipyToTensor(X)
        X = X.to_dense()
      except:
        X = X


      if(torch.cuda.is_available()):
        # print("yes")
        global adj
        X_tilde = X_tilde.cuda()
        C = C.cuda()
        theta = theta.cuda()
        X = X.cuda()
        J = J.cuda()
        zeros = zeros.cuda()
        ones = ones.cuda()
        eye = eye.cuda()
        try:
          adj = adj.cuda()
        except:
          pass

      def update(X_tilde,C,i):
          global L,adj
          thetaC = theta@C
          CT = torch.transpose(C,0,1)
          X_tildeT = torch.transpose(X_tilde,0,1)
          CX_tilde = C@X_tilde
          t1 = CT@thetaC + J
          term_bracket = torch.linalg.pinv(t1)
          thetacX_tilde = thetaC@(X_tilde)

          L = 1/k

          t1 = -2*gamma_param*(thetaC@term_bracket)
          t2 = alpha_param*(CX_tilde-X)@(X_tildeT)
          t3 = 2*thetacX_tilde@(X_tildeT)
          t4 = lambda_param*(C@ones)
          try:
            try:
              C = C.cuda()
            except:
              pass
            adj = torch.tensor(adj).cuda()
          except:
            pass
          t5 = 2*beta_param*(adj@C)
          T2 = (t1+t2+t3+t4+t5)/L
          Cnew = (C-T2).maximum(zeros)
          t1 = CT@thetaC*(2/alpha_param)
          t2 = CT@C
          t1 = torch.linalg.pinv(t1+t2)
          t1 = t1@CT
          t1 = t1@X
          X_tilde_new = t1
          Cnew[Cnew<thresh] = thresh
          for i in range(len(Cnew)):
              Cnew[i] = Cnew[i]/torch.linalg.norm(Cnew[i],1)
          for i in range(len(X_tilde_new)):
            X_tilde_new[i] = X_tilde_new[i]/torch.linalg.norm(X_tilde_new[i],1)
          return X_tilde_new,Cnew


      for i in tqdm(range(30)):
          X_tilde,C = update(X_tilde,C,i)

      return X_tilde,C




In [18]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(X.shape[1], 64)
        self.conv2 = GCNConv(64, NO_OF_CLASSES)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, x, edge_index):

        try:
          x = x.cuda()
        except:
          pass
        try:
          edge_index = edge_index.cuda()
        except:
          pass
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)



In [19]:
from torch_geometric.nn import GATConv
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 64
        self.in_head = 64
        self.out_head = 32

        self.conv1 = GATConv(X.shape[1], self.hid, heads=self.in_head, dropout=0.2)
        self.conv2 = GATConv(self.hid*self.in_head, NO_OF_CLASSES, concat=False, heads=self.out_head, dropout=0.2)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()


    def forward(self, x,edge_index):

        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)


In [20]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import APPNP

class APPNPNet(torch.nn.Module):
    def __init__(self):
        super(APPNPNet, self).__init__()
        self.lin1 = Linear(X.shape[1], 128)
        self.lin2 = Linear(128,NO_OF_CLASSES)
        self.prop1 = APPNP(16, 0.1)

    def reset_parameters(self):
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, x, edge_index):

        x = F.dropout(x, training=self.training)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, training=self.training)
        x = self.lin2(x)
        x = self.prop1(x, edge_index)

        return F.log_softmax(x, dim=1)

In [21]:
from random import sample

In [22]:
def get_accuracy(C_0,L,X_t_0):
    global labels, NO_OF_CLASSES,k
    t=[]
    for i in [1,2,3,4,5,6,7,8,9,10]:
        C_0_new=np.zeros(C_0.shape)
        for i in range(C_0.shape[0]):
            C_0_new[i][np.argmax(C_0[i])]=1
        # print(C_0_new)
        # C_0_new=C_0
        from scipy import sparse
        #Lc=C_0.T@L@C_0
        Lc=C_0_new.T@L@C_0_new
        # print("L:", Lc.shape)
        # Lc=L_new
        #print(Lc)
        Wc=(-1*Lc)*(1-np.eye(Lc.shape[0]))
        # print("W:", Wc.shape)
        Wc[Wc<0.1]=0
        Wc=sparse.csr_matrix(Wc)
        Wc = Wc.tocoo()
        row = torch.from_numpy(Wc.row).to(torch.long)
        col = torch.from_numpy(Wc.col).to(torch.long)
        edge_index_coarsen2 = torch.stack([row, col], dim=0)
        #print("edgecoarsen:", edge_index_coarsen2.shape)
        edge_weight = torch.from_numpy(Wc.data)
        #print("edgeweight:", edge_weight.shape)
        def one_hot(x, class_count):
            return torch.eye(class_count)[x, :]

        device = torch.device('cpu')
        labels=labels
        Y = labels
        #print("Y:", Y.shape)
        Y = one_hot(Y,NO_OF_CLASSES)
        # NO_OF_CLASSES=Y.shape[1]
        P=np.linalg.pinv(C_0_new)
        labels_coarse = torch.argmax(torch.sparse.mm(torch.Tensor(P).double() , Y.double()).double() , 1)
        #print("Lables:", labels_coarse.shape)

        #torch.Tensor(C2)@X
        Wc=Wc.toarray()
        #Wc[Wc<0.01]=0
        C2=np.linalg.pinv(C_0_new)
        model = Net().to(device)
        device = torch.device('cpu')
        lr=0.01
        decay=0.0001
        try:
          X=np.array(features.todense())
        except:
          X = np.array(features)
        #print("X:",X.shape)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=decay)
        # criterion=torch.nn.CrossEntropyLoss()
        x=sample(range(0, int(k)), k)

        from datetime import datetime
        Xt=P@X
        # Xt=X_t_0
        def train():
            model.train()
            optimizer.zero_grad()
            out = model(torch.Tensor(Xt).to(device),edge_index_coarsen2)
            loss = F.nll_loss(out[x], labels_coarse[x])
            loss.backward()
            optimizer.step()
            return loss
        now1 = datetime.now()
        losses=[]
        for epoch in range(150):
            loss=train()
            losses.append(loss)
            if(epoch%100==0):
                print(f'Epoch: {epoch:03d},loss: {loss:.4f}')
        now2 = datetime.now()
        pred=model(torch.Tensor(Xt).to(device),edge_index_coarsen2).argmax(dim=1)
        def train_accuracy():
            model.eval()
            correct = (pred[x] == labels_coarse[x]).sum()
            acc = int(correct) /len(x)
            return acc

        t+=[(now2-now1).total_seconds()]

        zz=sample(range(0, int(NO_OF_NODES)), NO_OF_NODES)
        global adj
        try:
          adj = adj.detach().cpu().numpy()
        except:
          try:
            adj = adj.numpy()
          except:
            adj = adj
        Wc=sparse.csr_matrix(adj)
        Wc = Wc.tocoo()
        row = torch.from_numpy(Wc.row).to(torch.long)
        col = torch.from_numpy(Wc.col).to(torch.long)
        edge_index_coarsen = torch.stack([row, col], dim=0)
        edge_weight = torch.from_numpy(Wc.data)
        pred=model(torch.Tensor(X),edge_index_coarsen).argmax(dim=1)
        pred=np.array(pred)
        correct =(pred[zz]==labels[zz]).sum()
        acc = int(correct) /NO_OF_NODES
        return acc



In [23]:
from scipy import sparse
from datetime import datetime

def get_accuracy1(C_0,L,X_t_0,NO_OF_EPOCHS=100):
    global labels, NO_OF_CLASSES,k,X
    t=[]
    for i in range(1,11):
        C_0_new=torch.zeros(C_0.shape)

        try:
          L = torch.tensor(L).cuda()
        except:
          try:
            L = torch.tensor(L)
          except:
            L = L

        try:
          C_0_new = C_0_new.cuda()
        except:
          C_0_new = C_0_new


        for i in range(C_0.shape[0]):
            C_0_new[i][torch.argmax(C_0[i])] = 1

        Lc = C_0_new.T@L@C_0_new
        Wc = (-1*Lc)*(1-torch.eye(Lc.shape[0]).cuda())
        Wc[Wc<0.1] = 0
        Wc = Wc.cpu().detach().numpy()
        Wc = sparse.csr_matrix(Wc)
        Wc = Wc.tocoo()
        row = torch.from_numpy(Wc.row).to(torch.long)
        col = torch.from_numpy(Wc.col).to(torch.long)
        edge_index_coarsen2 = torch.stack([row, col], dim=0)
        edge_weight = torch.from_numpy(Wc.data)

        def one_hot(x, class_count):
            xtemp = x.detach().cpu()
            return torch.eye(class_count)[xtemp, :].cuda()

        try:
          edge_index_coarsen2 = edge_index_coarsen2.cuda()
        except:
          pass

        try:
          labels = labels.cuda()
        except:
          try:
            labels = torch.tensor(labels).cuda()
          except:
            try:
              labels = torch.tensor(labels)
            except:
              pass

        Y = labels
        Y = one_hot(Y,NO_OF_CLASSES)

        P = torch.linalg.pinv(C_0_new)
        labels_coarse = torch.argmax(torch.sparse.mm(torch.Tensor(P).double() , Y.double()).double() , 1)

        Wc = Wc.toarray()
        model = Net().to(device)

        lr=0.01
        decay=0.0001
        try:
          X = torch.tensor(features.todense())
        except:
          X = torch.tensor(features)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=decay)
        x=sample(range(0, int(k)), k)


        try:
          P = P.cuda()
        except:
          pass


        X1 = X

        try:
          X1 = torch.tensor(X1)
          try:
            X1 = X1.cuda()
          except:
            pass
        except:
          pass

        Xt = P@X1

        best_val_acc = 0.0
        best_test_acc = 0.0

        data.y = data.y.to(device)
        data.train_mask = data.train_mask.to(device)
        data.val_mask = data.val_mask.to(device)
        data.test_mask = data.test_mask.to(device)

        def train():
            model.train()
            optimizer.zero_grad()
            out = model(torch.Tensor(Xt).to(device),edge_index_coarsen2)
            loss = F.nll_loss(out[x], labels_coarse[x])
            loss.backward()
            optimizer.step()
            return loss

        now1 = datetime.now()

        losses=[]

        for epoch in range(NO_OF_EPOCHS):
            loss = train()
            losses.append(loss)
            model.eval()
            with torch.no_grad():
            # Forward pass on the original graph for validation accuracy
              valid_out = model(data.x.to(device), data.edge_index.to(device))
              valid_acc = (valid_out[data.val_mask].argmax(dim=1) == data.y[data.val_mask]).float().mean()

            # If validation accuracy improves, update best validation and test accuracy
            if valid_acc > best_val_acc:
                best_val_acc = valid_acc
                test_out = model(data.x.to(device), data.edge_index.to(device))
                test_acc = (test_out[data.test_mask].argmax(dim=1) == data.y[data.test_mask]).float().mean()
                best_test_acc = test_acc
                print(f'Epoch: {epoch}, Train Loss: {loss.item()}, Valid Acc: {valid_acc.item()}, Test Acc: {test_acc.item()}')

            if(epoch%100==0):
                print(f'Epoch: {epoch:03d},loss: {loss:.4f}')
        print(f'Best Validation Accuracy: {best_val_acc.item()}, Best Test Accuracy: {best_test_acc.item()}')

        now2 = datetime.now()
        pred=model(torch.Tensor(Xt).to(device),edge_index_coarsen2).argmax(dim=1)

        def train_accuracy():
            model.eval()
            correct = (pred[x] == labels_coarse[x]).sum()
            acc = int(correct) /len(x)
            return acc

        t+=[(now2-now1).total_seconds()]

        zz=sample(range(0, int(NO_OF_NODES)), NO_OF_NODES)
        global adj
        try:
          adj = adj.detach().cpu().numpy()
        except:
          try:
            adj = adj.numpy()
          except:
            adj = adj

        Wc = sparse.csr_matrix(adj)
        Wc = Wc.tocoo()
        row = torch.from_numpy(Wc.row).to(torch.long)
        col = torch.from_numpy(Wc.col).to(torch.long)
        edge_index_coarsen = torch.stack([row, col], dim=0)
        edge_weight = torch.from_numpy(Wc.data)

        try:
          edge_index_coarsen = edge_index_coarsen.cuda()
        except:
          pass

        pred = model(torch.Tensor(X),edge_index_coarsen).argmax(dim=1)
        try:
          pred = torch.tensor(pred)
        except:
          pass

        correct = (pred[zz] == labels[zz]).sum()
        acc = int(correct) /NO_OF_NODES
        print(f'Train Accuracy = {acc}')
        return acc,best_val_acc,best_test_acc



In [24]:
import time

In [25]:
import seaborn as sns
import matplotlib.pylab as plt
        # sns.heatmap(C_0.T@C_0)


#0.001 0.0001 1 0.0001
for lambda_param in [0.0001,]:
  for beta_param in [0.0001,]:
    for alpha_param in [0.0001,]:
      for gamma_param in [0.0001]:

        av = []

        start = time.time()

        for _ in range(1):

          # start = time.time()

          X_tilde = random(k, n, density=0.15, random_state=1, data_rvs=temp2.rvs)
          C = random(p, k, density=0.15, random_state=1, data_rvs=temp2.rvs)

          # try:

          X_t_0,C_0 = experiment(lambda_param,beta_param,alpha_param,gamma_param,C,X_tilde,theta,X)
          L = theta
          # C_0 = C_0.cpu().detach().numpy()
          # X_t_0 = X_t_0.cpu().detach().numpy()
          C_t_0 = C_0.T
          # try:
            # L = L.cpu().detach().numpy()
          # except:
            # L = L
          acc = get_accuracy1(C_0,L,X_t_0)
          end = time.time()
          print((end-start))
          av.append(acc)
          print("Accuracy = " + str(acc) + " " + str(lambda_param)+" " + str(beta_param)+" "+str(alpha_param)+" "+str(gamma_param))

          # except:
          #   print("Some Error Occurred")






<ipython-input-13-c0f51d08421d>:13: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:651.)
  return torch.sparse.FloatTensor(i, v, torch.Size(shape))
100%|██████████| 30/30 [00:04<00:00,  6.21it/s]
<ipython-input-23-19a9d0e451da>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  L = torch.tensor(L).cuda()
<ipython-input-23-19a9d0e451da>:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X1 = torch.tensor(X1)


Epoch: 0, Train Loss: 1.9855694770812988, Valid Acc: 0.29629629850387573, Test Acc: 0.3492647111415863
Epoch: 000,loss: 1.9856
Epoch: 5, Train Loss: 1.3911038637161255, Valid Acc: 0.29999998211860657, Test Acc: 0.3492647111415863
Epoch: 8, Train Loss: 1.1960428953170776, Valid Acc: 0.31111109256744385, Test Acc: 0.3602941334247589
Epoch: 9, Train Loss: 1.167938232421875, Valid Acc: 0.3185185194015503, Test Acc: 0.3639705777168274
Epoch: 10, Train Loss: 1.1879602670669556, Valid Acc: 0.32222220301628113, Test Acc: 0.36764705181121826
Epoch: 18, Train Loss: 0.8510102033615112, Valid Acc: 0.32592591643333435, Test Acc: 0.3639705777168274
Epoch: 19, Train Loss: 0.8306102156639099, Valid Acc: 0.33703702688217163, Test Acc: 0.37132352590560913
Epoch: 20, Train Loss: 0.7849332094192505, Valid Acc: 0.35185185074806213, Test Acc: 0.3860294222831726
Epoch: 21, Train Loss: 0.7440491914749146, Valid Acc: 0.38148146867752075, Test Acc: 0.40441176295280457
Epoch: 22, Train Loss: 0.7327563166618347, 

<ipython-input-23-19a9d0e451da>:174: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = torch.tensor(pred)


In [26]:
import itertools
import time
import numpy as np

# Define ranges for hyperparameters
lambda_params = [0.01,0.1,1,10,100]
beta_params = [0.01,0.1,1,10,100]
alpha_params = [0.01,0.1,1,10,100]
gamma_params = [0.01,0.1,1,10,100]

# Track the best parameters and results
best_val_accuracy = 0
best_test_accuracy = 0
best_params = None

# Perform Grid Search
for lambda_param, beta_param, alpha_param, gamma_param in itertools.product(
    lambda_params, beta_params, alpha_params, gamma_params
):
    print(f"Testing combination: λ={lambda_param}, β={beta_param}, α={alpha_param}, γ={gamma_param}")

    try:
        start = time.time()

        # Generate random data
        X_tilde = random(k, n, density=0.15, random_state=1, data_rvs=temp2.rvs)
        C = random(p, k, density=0.15, random_state=1, data_rvs=temp2.rvs)

        X_t_0,C_0 = experiment(lambda_param,beta_param,alpha_param,gamma_param,C,X_tilde,theta,X)
        L = theta
          # C_0 = C_0.cpu().detach().numpy()
          # X_t_0 = X_t_0.cpu().detach().numpy()
        C_t_0 = C_0.T


        # Compute accuracies
        train_acc,val_acc,test_acc = get_accuracy1(C_0,L,X_t_0)
        end = time.time()
        print((end-start))
        av.append(acc)



        print(f"Training Accuracy: {train_acc:.4f}, Validation Accuracy: {val_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Update best parameters if validation accuracy improves
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            best_test_accuracy = test_acc
            best_params = {
                "lambda_param": lambda_param,
                "beta_param": beta_param,
                "alpha_param": alpha_param,
                "gamma_param": gamma_param
            }

        end = time.time()
        print(f"Time Taken: {end - start:.2f} seconds\n")

    except Exception as e:
        print(f"Error for λ={lambda_param}, β={beta_param}, α={alpha_param}, γ={gamma_param}: {e}")

# Print the best results
print("\nBest Hyperparameters:")
print(best_params)
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Test Accuracy Corresponding to Best Validation Accuracy: {best_test_accuracy:.4f}")

Testing combination: λ=0.01, β=0.01, α=0.01, γ=0.01


100%|██████████| 30/30 [00:03<00:00,  9.76it/s]
<ipython-input-23-19a9d0e451da>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  L = torch.tensor(L).cuda()
<ipython-input-23-19a9d0e451da>:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X1 = torch.tensor(X1)


Epoch: 0, Train Loss: 1.9454900026321411, Valid Acc: 0.29629629850387573, Test Acc: 0.3492647111415863
Epoch: 000,loss: 1.9455


<ipython-input-23-19a9d0e451da>:174: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = torch.tensor(pred)


Best Validation Accuracy: 0.29629629850387573, Best Test Accuracy: 0.3492647111415863
Train Accuracy = 0.2848568790397045
4.412356376647949
Training Accuracy: 0.2849, Validation Accuracy: 0.2963, Test Accuracy: 0.3493
Time Taken: 4.41 seconds

Testing combination: λ=0.01, β=0.01, α=0.01, γ=0.1


...

Epoch: 0, Train Loss: 1.9460158348083496, Valid Acc: 0.29629629850387573, Test Acc: 0.3492647111415863
Epoch: 000,loss: 1.9460
Best Validation Accuracy: 0.29629629850387573, Best Test Accuracy: 0.3492647111415863
Train Accuracy = 0.2788550323176362
4.455545902252197
Training Accuracy: 0.2789, Validation Accuracy: 0.2963, Test Accuracy: 0.3493
Time Taken: 4.46 seconds

Testing combination: λ=100, β=100, α=100, γ=10


100%|██████████| 30/30 [00:03<00:00,  8.06it/s]


Epoch: 0, Train Loss: 1.945935606956482, Valid Acc: 0.29629629850387573, Test Acc: 0.34191176295280457
Epoch: 000,loss: 1.9459
Best Validation Accuracy: 0.29629629850387573, Best Test Accuracy: 0.34191176295280457
Train Accuracy = 0.2830101569713758
4.766242504119873
Training Accuracy: 0.2830, Validation Accuracy: 0.2963, Test Accuracy: 0.3419
Time Taken: 4.77 seconds

Testing combination: λ=100, β=100, α=100, γ=100


100%|██████████| 30/30 [00:03<00:00,  9.46it/s]


Epoch: 0, Train Loss: 1.946036696434021, Valid Acc: 0.29629629850387573, Test Acc: 0.3492647111415863
Epoch: 000,loss: 1.9460
Best Validation Accuracy: 0.29629629850387573, Best Test Accuracy: 0.3492647111415863
Train Accuracy = 0.2677746999076639
4.223769426345825
Training Accuracy: 0.2678, Validation Accuracy: 0.2963, Test Accuracy: 0.3493
Time Taken: 4.22 seconds


Best Hyperparameters:
{'lambda_param': 10, 'beta_param': 0.01, 'alpha_param': 100, 'gamma_param': 10}
Best Validation Accuracy: 0.8370
Test Accuracy Corresponding to Best Validation Accuracy: 0.7794
